In [1]:

%matplotlib tk
import clock
from clock import solve_ivp, pendulum
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path

In [2]:
def plot_pendulum():

    def time_series_plot(init):
        t = np.linspace(0, 4*np.pi, 500)
        sol = solve_ivp(pendulum, (t[0], t[-1]), init, t_eval=t)

        plt.plot(sol.t, sol.y[0,:], label=r'$\theta$')
        plt.plot(sol.t, sol.y[1,:], label=r'$\dot{\theta}$')

    time_series_plot([2, 0])
    plt.yticks(np.arange(-2, 3))
    plt.xticks(np.arange(5) * np.pi, ['$0$', '$\pi$', *[f'${x}\pi$' for x in np.arange(2, 5)]])
    plt.tick_params(which='both', left=False, bottom=False)
    plt.legend()
    plt.savefig('test.png', dpi=600, facecolor='white', bbox_inches='tight')


In [3]:
matplotlib.style.use('ggplot')
plot_pendulum()

In [81]:
init=[0,0.7]
t = np.linspace(0, 2*np.pi, 200)
sol = solve_ivp(pendulum, (t[0], t[-1]), init, t_eval=t)

In [125]:
from matplotlib import animation
from matplotlib.lines import Line2D
from matplotlib.patches import Circle, FancyArrow

BGCOLOR = "#e9f2cb"

fig, axes = plt.subplots(ncols=2)
fig.tight_layout(pad=0)
fig.set_size_inches(6,3)
ax = axes[0]
diagram = axes[1]



pendulum = Pendulum(diagram)
trajectory = Trajectory(ax, sol)

def update(pos):
    trajectory.set_marker(pos)
    pendulum.theta = pos[0]
            
fig.set_facecolor(BGCOLOR)


ani = animation.FuncAnimation(fig, update, trajectory, interval=20, blit=False)
ani.save_count = (sol.y.shape[1])
#ani.save('test.gif', writer='imagemagick', fps=50, dpi=100, savefig_kwargs=dict(facecolor=BGCOLOR))
plt.show()

In [126]:
import matplotlib.axes
from typing import Any, Union, Iterable, Text

class EqualAspectAxis:
    """Configures axis to be suitable for rendering.
    
    The output is rendered by creating patches directly.
    To make this look nice, we want a specified set of
    boundaries for the plotting area, an equal aspect
    ratio (so things don't look stretched) and to remove
    the axis labels / lines.
    """
    def __init__(self, ax):
        ax.axis((-1.5, 1.5, -1.5, 1.5))
        ax.set_aspect('equal')
        ax.set_axis_off()

        
class Pendulum(EqualAspectAxis):
    """Draws a pendulum onto a matplotlib axis.
    
    Examples:
    
        >>> fig, ax = plt.subplots()
        >>> pendulum = Pendulum(ax)
        >>> pendulum.theta = 0.1
        >>> plt.show()
        
    Arguments:
    
        ax: Axis to be drawn on
        color: 
            Color specification (hex, array etc as 
            accepted by matplotlib) for the pendulum.
            
    """
    def __init__(self, ax: matplotlib.axes.Axes, color: Union[Text, Iterable]='#f086dc'):
        super().__init__(ax)
        self.ax = ax
        self.color = color
        self._theta = 0.0
        
        self._bar = Line2D([0], [0], lw=5, color=self.color)
        self._weight = Circle(xy=(0,0), radius=0.2, facecolor=self.color)
        self.ax.add_line(self._bar)
        self.ax.add_patch(self._weight)
        
    @property
    def theta(self):
        """Angle of the pendulum."""
        return self._theta
    
    @theta.setter
    def theta(self, value):
        """Set the pendulum angle."""
        self._theta = value
        vector = np.array([
            [0, 0],
            [np.sin(self._theta), -np.cos(self._theta)]
        ])
        vector *= 1.5
        vector[:,1] += 0.5
        self._bar.set_xdata(vector[:,0])
        self._bar.set_ydata(vector[:,1])
        self._weight.set_center(vector[1,:])
        
        
class Trajectory(EqualAspectAxis):
    """Draw the state space trajectory of the system.
    
    This renders a plot of the whole trajectory (as a line
    plot) as well as a current position marker at the
    given time t.
    
    Example:
    
        >>> # Obtain a solution object from solve_ivp
        >>> solution = scipy.integrate.solve_ivp(...)
        >>> fig, ax = plt.subplots()
        >>> trajectory = Trajectory(ax, solution)
        >>> # Set the marker to show the neupdatext position of the solution
        >>> trajectory.step()
        array([0.0, 0.9])
        >>> plt.show()
        
    Arguments:
    
        ax: Axis to draw on
        solution: Trajectory data to plot
        
    """
    
    def __init__(self, ax: matplotlib.axes.Axes, solution: Any, 
                 color: Union[Text, Iterable]='#f086dc', radius: int = 0.1):
        super().__init__(ax)
        self.ax = ax
        self.solution = solution
        self.color = color
        
        # Add arrows to represent axis labels
        xarrow = FancyArrow(-1.31, -1.3, 0.3, 0, width=0.02, edgecolor='k', facecolor='k')
        ax.text(-0.8, -1.3, r'$\theta$', fontsize='x-large', verticalalignment='center')
        ax.add_patch(xarrow)

        yarrow = FancyArrow(-1.3, -1.31, 0, 0.3, width=0.02, edgecolor='k', facecolor='k')
        ax.text(-1.3, -0.8, r'$\dot{\theta}$', fontsize='x-large', horizontalalignment='center')
        ax.add_patch(yarrow)
        
        # Add the marker to the axis
        self._marker = Circle(xy=self.solution.y[:,0], radius=radius, facecolor=color)
        ax.add_patch(self._marker)

        ax.plot(sol.y[0,:], sol.y[1,:], color=color)

    def __call__(self):
        for y in self.solution.y.T[1:,:]:
            yield y
        
    def set_marker(self, position):
        self._marker.set_center(position)
        

In [116]:
import matplotlib.axis
isinstance(ax, matplotlib.axis.Axis)

False

In [112]:
fix, ax = plt.subplots()
p = Pendulum(ax)
p.theta=0.1
plt.show()

In [120]:
sol.__class__

scipy.integrate._ivp.ivp.OdeResult

In [118]:
from scipy.integrate import OdeSolution